In [1]:
import torch
import math

In [17]:
!pip uninstall pyarrow -y
!pip install pyarrow

Found existing installation: pyarrow 6.0.1
Uninstalling pyarrow-6.0.1:
  Successfully uninstalled pyarrow-6.0.1
  Using cached pyarrow-11.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.1 MB)


In [1]:
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
pa.__version__

'11.0.0'

In [3]:
# читаем все файлы из папки
import glob
parquet_files = glob.glob("../../mts/data/competition_data_final_pqt/*.parquet")
par = None
for file in parquet_files:
    print(file)
    par1 = pq.read_table(file)
    if par is None:
        par = par1
    else:
        par = pa.concat_tables([par, par1])

../../mts/data/competition_data_final_pqt/part-00000-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet
../../mts/data/competition_data_final_pqt/part-00001-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet
../../mts/data/competition_data_final_pqt/part-00002-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet
../../mts/data/competition_data_final_pqt/part-00003-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet
../../mts/data/competition_data_final_pqt/part-00004-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet
../../mts/data/competition_data_final_pqt/part-00005-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet
../../mts/data/competition_data_final_pqt/part-00006-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet
../../mts/data/competition_data_final_pqt/part-00007-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet
../../mts/data/competition_data_final_pqt/part-00008-aba60f69-2b63-4cc1-95ca-542598094698-c000.snappy.parquet
../../mts/

In [4]:
par.shape[0]

322899435

In [5]:
par.column_names

['region_name',
 'city_name',
 'cpe_manufacturer_name',
 'cpe_model_name',
 'url_host',
 'cpe_type_cd',
 'cpe_model_os_type',
 'price',
 'date',
 'part_of_day',
 'request_cnt',
 'user_id']

In [ ]:
# делаем энкодинг, если надо
from sklearn.preprocessing import LabelEncoder
cnl = LabelEncoder()
cnl.fit(par['city_name'])

url = LabelEncoder()
url_enc = url.fit_transform(par['url_host'])

In [ ]:
# добавим столбец, это оказывается просто
par = par.append_column('url_enc', pa.array(url_enc))

In [ ]:
par.column_names

In [11]:
import torch
import time

BATCH_SIZE = 1024

#собираем батчи для обучения, с векторами чуть посложнее, но все в рамках простого
st = time.perf_counter()

for i, batch in enumerate(par.to_batches(max_chunksize = BATCH_SIZE)):
    city_name = cnl.transform(batch['city_name'])
    url_host  = url.transform(batch['url_host'])
    city_name = torch.tensor(city_name, dtype=torch.int16)
    url_host  = torch.tensor(url_host , dtype=torch.int32)
    ret_batch = {
            'city_name' : city_name, 
            'url_host'  : url_host,
            }
    print(f"batch = {i:03}, size = {len(city_name)}")
    print(ret_batch)
    if len(city_name) < BATCH_SIZE:
        pass
    if i > 10:
        break
    # print(ret_batch)
et = time.perf_counter()

print(f"{et-st} sec")

batch = 000, size = 1024
{'city_name': tensor([409, 409, 409,  ..., 409, 409, 409], dtype=torch.int16), 'url_host': tensor([ 5788, 12900, 17626,  ..., 66084, 66084, 89341], dtype=torch.int32)}
batch = 001, size = 1024
{'city_name': tensor([409, 409, 409,  ..., 732, 732, 732], dtype=torch.int16), 'url_host': tensor([ 91266,   8960,  12900,  ...,  44040, 160875, 160876],
       dtype=torch.int32)}
batch = 002, size = 1024
{'city_name': tensor([732, 732, 732,  ..., 732, 732, 732], dtype=torch.int16), 'url_host': tensor([160876, 174759, 180836,  ...,   5790,  49054, 111474],
       dtype=torch.int32)}
batch = 003, size = 1024
{'city_name': tensor([732, 732, 311,  ..., 592, 592, 592], dtype=torch.int16), 'url_host': tensor([180836, 194523, 194523,  ..., 111473, 157450, 171045],
       dtype=torch.int32)}
batch = 004, size = 1024
{'city_name': tensor([592, 592, 592,  ..., 592, 592, 592], dtype=torch.int16), 'url_host': tensor([  6318,  17626,  59366,  ..., 140099, 197762,  17626],
       dty